# Создание дашборда по пользовательским событиям для агрегатора новостей


### Описание проекта

Вы работаете аналитиком в Яндекс.Дзене. Почти всё ваше время занимает анализ пользовательского взаимодействия с карточками статей.

Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия». А пользователей системы характеризует возрастная категория.

Есть три способа взаимодействия пользователей с системой:

- Карточка отображена для пользователя (show);

- Пользователь кликнул на карточку (click);

- Пользователь просмотрел статью карточки (view).

Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы:

1. Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
2. Как много карточек генерируют источники с разными темами?
3. Как соотносятся темы карточек и темы источников?

На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.

Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан для вас дата-инженерами.

### ТЗ для создания дашборда

**Бизнес-задача**: анализ взаимодействия пользователей с карточками Яндекс.Дзен;

**Насколько часто предполагается пользоваться дашбордом:** не реже, чем раз в неделю;

**Кто будет основным пользователем дашборда:** менеджеры по анализу контента;

**Состав данных для дашборда:**

- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);

- Разбивка событий по темам источников;

- Таблица соответствия тем источников темам карточек;

**По каким параметрам данные должны группироваться:**

- Дата и время;

- Тема карточки;

- Тема источника;

- Возрастная группа;

- Характер данных:

История событий по темам карточек — абсолютные величины с разбивкой по минутам;

Разбивка событий по темам источников — относительные величины (% событий);

Соответствия тем источников темам карточек - абсолютные величины;

**Важность:** все графики имеют равную важность;

**Источники данных для дашборда:** дата-инженеры обещали подготовить для вас агрегирующую таблицу dash_visits. **Вот её структура:**

- record_id — первичный ключ,

- item_topic — тема карточки,

- source_topic — тема источника,

- age_segment — возрастной сегмент,

- dt — дата и время,

- visits — количество событий.


**Макет дашборда:**

![Макет дашборда](https://disk.yandex.ru/i/367cV_Ef8kA6Sg)

Если фото не прогрузилось, макет можно посмотреть по ссылке: https://disk.yandex.ru/i/367cV_Ef8kA6Sg

**Приконнектимся к базе, где хранятся нужные данные**

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student',
            'pwd': 'Sdf4$2;d-d30pp',
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432,
            'db': 'data-analyst-zen-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

query = ''' SELECT *
            FROM dash_visits
        '''
dash_visits = pd.io.sql.read_sql(query, con = engine)

**Ознакомимся с данными**

In [2]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [3]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


**Посмотрим сколько есть тем карточек**

In [4]:
dash_visits['item_topic'].value_counts()

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

**Посмотрим относительное количество тем карточек**

In [5]:
item_topic_counts = dash_visits['item_topic'].value_counts()

total_items = item_topic_counts.sum()

item_topic_percents = item_topic_counts / total_items * 100

print(item_topic_percents.round(2).astype(str) + '%')

Отношения              5.0%
Интересные факты      4.99%
Наука                  4.9%
Подборки              4.74%
Полезные советы       4.63%
Общество              4.63%
Россия                 4.5%
История               4.43%
Семья                 4.19%
Путешествия           4.06%
Деньги                4.01%
Женщины                4.0%
Дети                   4.0%
Туризм                3.92%
Здоровье              3.91%
Красота               3.88%
Культура              3.77%
Юмор                  3.67%
Искусство             3.64%
Рассказы              3.61%
Психология            3.43%
Скандалы              3.33%
Знаменитости          3.17%
Женская психология    2.97%
Шоу                   2.62%
Name: item_topic, dtype: object


**Посмотрим сколько есть тем источников**

In [6]:
dash_visits['source_topic'].value_counts()

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

**Посмотрим относительное количество тем источников**

In [7]:
total_sources = dash_visits['source_topic'].nunique()

source_topic_counts = dash_visits['source_topic'].value_counts(normalize=True) * 100

source_topic_counts = source_topic_counts.round(2)

print(source_topic_counts.round(2).astype(str) + '%')

Семейные отношения    5.93%
Россия                5.49%
Знаменитости          5.37%
Полезные советы       5.13%
Путешествия           5.08%
Кино                   4.9%
Дети                  4.75%
История               4.67%
Семья                 4.57%
Одежда                4.49%
Здоровье              4.04%
Искусство             3.99%
Авто                   3.5%
Психология            3.43%
Сад и дача            3.37%
Политика              3.33%
Спорт                 3.28%
Сделай сам            3.24%
Ремонт                 3.2%
Деньги                3.16%
Еда                   2.97%
Интерьеры             2.63%
Строительство         2.47%
Музыка                2.44%
Технологии            2.41%
Финансы               2.17%
Name: source_topic, dtype: object


**Посмотрим на возрастные группы**

In [8]:
dash_visits['age_segment'].value_counts()

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

**Проверим за какой период данные** 

In [9]:
print('Мин:', dash_visits['dt'].min())
print('Макс:',dash_visits['dt'].max())

Мин: 2019-09-24 18:28:00
Макс: 2019-09-24 19:00:00


**Выгрузим данные**

In [10]:
dash_visits.to_csv('dash_visits.csv', index=False)

## Сылка на дашборд в Tableau: 
https://public.tableau.com/app/profile/kirill.naumenkov/viz/Yandex-Dzen_Naumenkov/-_1?publish=yes

## Ссылка на презентацию на Яндекс Диск: 
https://disk.yandex.ru/i/8BBVC1sqi_BYMw